In [8]:
from src.inference import evaluate_model

In [53]:

def interpret(model_path):
    cer_val, wer_val = evaluate_model(
    model_path=model_path, 
    datapath= model_path + '/val.csv' 
    )
    cer_test, wer_test = evaluate_model(
    model_path=model_path,
    datapath='data/testset.npy'
    )
    print("\t\tCER\t\tWER")
    #round up to 4 decimal places 
    cer_val = round(cer_val, 4)
    wer_val = round(wer_val, 4)
    cer_test = round(cer_test, 4)
    wer_test = round(wer_test, 4)
    
    print(f"Validation\t{cer_val}\t\t{wer_val}")
    print(f"Test\t\t{cer_test}\t\t{wer_test}")
    
    return (cer_val, wer_val, cer_test, wer_test)

### HTR NET  without the custom dataset

In [54]:
htr_net_no_cus = interpret('results/htr_net_no_custom_new_split')


100%|██████████| 247/247 [00:02<00:00, 99.47it/s] 

		CER		WER
Validation	0.152		0.29
Test		0.3601		0.6073


### HTR NET with the custom dataset

In [58]:
htr_net_cus = interpret('results/htr_full_data')

100%|██████████| 247/247 [00:02<00:00, 98.91it/s]

		CER		WER
Validation	0.091		0.2004
Test		0.2021		0.3846


### CNN-BILSTM from mltu WITHOUT custom dataset

In [63]:
cnn_bilstm_no_cust = interpret('results/cnn_bilstm_mltu1')


  0%|          | 0/1552 [00:00<?, ?it/s]

100%|██████████| 247/247 [00:01<00:00, 186.55it/s]

		CER		WER
Validation	0.1004		0.2313
Test		0.2399		0.413


### CNN-BILSTM from mltu WITH custom dataset

In [64]:
cnn_bilstm_cust = interpret('results/cnn_bilstm_mltu_all_data')

100%|██████████| 247/247 [00:01<00:00, 184.24it/s]

		CER		WER
Validation	0.0948		0.195
Test		0.1761		0.3279


### Summary

In [71]:
print("Custom dataset \t\t WITHOUT \t\t WITH")
print("\t\t\t Val \t Test \t\t Val \t Test") 
print(f"HTR-Net \t\t {htr_net_no_cus[0]} \t {htr_net_no_cus[2]} \t {htr_net_cus[0]} \t {htr_net_cus[2]}")
print(f"CNN-BiLSTM-MLTU  \t{cnn_bilstm_no_cust[0]} \t {cnn_bilstm_no_cust[2]} \t {cnn_bilstm_cust[0]}  {cnn_bilstm_cust[2]}")


Custom dataset 		 WITHOUT 		 WITH
			 Val 	 Test 		 Val 	 Test
HTR-Net 		 0.152 	 0.3601 	 0.091 	 0.2021
CNN-BiLSTM-MLTU  	0.1004 	 0.2399 	 0.0948  0.1761
